In [1]:
import os


os.environ['https_proxy'] = 'http://172.17.0.1:1081'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [2]:
from axolotl.cli import load_cfg


cfg = load_cfg('../examples/mlpspec/llama3-8b-instruct.yaml')
cfg

/root/work/axolotl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-11 16:09:59,399] [INFO] [axolotl.utils.config.models.input.check_eval_packing:991] [PID:1485860] [RANK:0] explicitly setting `eval_sample_packing` to match `sample_packing`
[2024-11-11 16:09:59,407] [INFO] [axolotl.normalize_config:183] [PID:1485860] [RANK:0] GPU memory usage baseline: 0.000GB (+1.184GB misc)


{'mlpspec': {'emb_dim': 4096,
  'inner_dim': 3072,
  'n_candidates': 5,
  'n_predict': 4,
  'top_k_tokens_per_head': [4, 3, 2, 2],
  'vocab_size': 128256},
 'medusa_logging': False,
 'medusa_scheduler': 'constant',
 'medusa_only_heads': False,
 'medusa_num_unfreeze_layers': 0,
 'medusa_distillation_regularization': 0,
 'medusa_self_distillation': False,
 'type_of_model': 'LlamaForCausalLM',
 'lisa_layers_attribute': 'model.layers',
 'wandb_project': 'mlpspec_test',
 'gradient_accumulation_steps': 4,
 'micro_batch_size': 2,
 'train_on_inputs': False,
 'group_by_length': False,
 'learning_rate': 0.0005,
 'weight_decay': 0.0,
 'optimizer': <OptimizerNames.ADAMW_BNB: 'adamw_bnb_8bit'>,
 'lr_scheduler': <SchedulerType.COSINE: 'cosine'>,
 'num_epochs': 4,
 'load_in_8bit': False,
 'load_in_4bit': False,
 'lora_dropout': 0.0,
 'qlora_sharded_model_loading': False,
 'loraplus_lr_embedding': 1e-06,
 'output_dir': './outputs/llama3-8b-instruct',
 'base_model': '/models/Meta-Llama-3-8B-Instruct',


In [3]:
from axolotl.prompt_strategies.sharegpt import register_llama3_template


register_llama3_template(cfg.default_system_message)

In [4]:
#from axolotl.utils.models import load_model, load_tokenizer


#tokenizer = load_tokenizer(cfg)
#tokenizer

In [5]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(cfg.base_model_config, use_fast=False)
tokenizer

PreTrainedTokenizerFast(name_or_path='/models/Meta-Llama-3-8B-Instruct', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|

In [6]:
from axolotl.utils.models import load_model


model, peft_config = load_model(cfg, tokenizer, inference=False)
model

[2024-11-11 16:10:00,440] [INFO] [accelerate.utils.modeling.get_balanced_memory:1005] [PID:1485860] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [01:01<00:00, 15.41s/it]


[2024-11-11 16:11:02,185] [INFO] [axolotl.load_model:841] [PID:1485860] [RANK:0] GPU memory usage after model load: 14.958GB (+0.126GB cache)
[2024-11-11 16:11:02,208] [INFO] [axolotl.load_model:906] [PID:1485860] [RANK:0] converting modules to torch.bfloat16 for flash attention
[2024-11-11 16:11:44,008] [WARNING] [axolotl.load_model:1120] [PID:1485860] [RANK:0] there are no parameters that require gradient updates


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    

In [7]:
ids = tokenizer.encode('hello, what are you going to', return_tensors="pt").to('cuda')
ids

tensor([[128000,  15339,     11,   1148,    527,    499,   2133,    311]],
       device='cuda:0')

In [8]:
out = model.model(ids, output_hidden_states=True)
out

BaseModelOutputWithPast(last_hidden_state=tensor([[[ 4.2188e+00, -2.0410e-01, -1.8438e+00,  ..., -2.8594e+00,
           1.3594e+00,  2.8906e-01],
         [-1.4141e+00, -7.1094e-01, -5.4688e-01,  ..., -8.2031e-01,
          -1.8672e+00,  1.6484e+00],
         [-1.7812e+00, -3.1406e+00,  3.7344e+00,  ..., -1.9062e+00,
          -1.3750e+00,  2.5312e+00],
         ...,
         [-9.4922e-01, -1.8828e+00,  3.0625e+00,  ...,  4.0000e+00,
          -1.8750e+00,  3.5625e+00],
         [-1.4531e+00,  1.1328e+00,  1.9453e+00,  ..., -1.4551e-01,
           7.1094e-01,  9.3750e-01],
         [-3.3750e+00, -9.7656e-02,  7.3047e-01,  ...,  7.8201e-04,
          -9.9609e-02,  2.2969e+00]]], device='cuda:0', dtype=torch.bfloat16), past_key_values=None, hidden_states=(tensor([[[-8.2970e-05,  2.5749e-04, -2.4605e-04,  ..., -3.2425e-04,
          -2.1553e-04,  4.7112e-04],
         [ 1.3123e-02, -1.8066e-02,  7.7438e-04,  ..., -7.6904e-03,
           7.2937e-03,  9.9487e-03],
         [-2.5558e-04, -4

In [9]:
out.last_hidden_state.shape

torch.Size([1, 8, 4096])

In [10]:
model.mlp_model

MLPSpeculatorPreTrainedModel(
  (speculator): MLPSpeculator(
    (emb): ModuleList(
      (0-3): 4 x Embedding(128256, 3072)
    )
    (proj): ModuleList(
      (0): Linear(in_features=4096, out_features=3072, bias=False)
      (1-3): 3 x Linear(in_features=3072, out_features=3072, bias=False)
    )
    (head): ModuleList(
      (0-3): 4 x Linear(in_features=3072, out_features=128256, bias=False)
    )
    (ln): ModuleList(
      (0-3): 4 x LayerNormParameterized()
    )
    (activation): GELU(approximate='none')
  )
)

In [11]:
speculator = model.mlp_model.speculator
len(speculator.emb)

4

In [12]:
z = speculator.emb[0](ids)
z.shape

torch.Size([1, 8, 3072])

In [13]:
state = speculator.proj[0](out.last_hidden_state)
state.shape

torch.Size([1, 8, 3072])

In [14]:
alpha = speculator.emb_weight / speculator.state_weight
alpha

17.047642905814858

In [15]:
import torch


state = torch.add(state, z, alpha=alpha)
state.shape

torch.Size([1, 8, 3072])

In [16]:
state = speculator.activation(speculator.ln[0](state))
state.shape

torch.Size([1, 8, 3072])

In [17]:
out = speculator.head[0](state)
out.shape

torch.Size([1, 8, 128256])

In [18]:
speculator.head[0].weight.shape

torch.Size([128256, 3072])

In [19]:
model.lm_head.weight.shape

torch.Size([128256, 4096])

---

In [ ]:
out2 = model.forward(ids, mlpspec_return=True)
out2

base_out.last_hidden_state.shape=torch.Size([1, 4, 4096])
0.state.shape=torch.Size([1, 0, 4096])
0.inds.shape=torch.Size([1, 4])
1.state.shape=torch.Size([1, 0, 4096])
2.state.shape=torch.Size([1, 0, 3072])
3.state.shape=torch.Size([1, 0, 3072])
4.state.shape=torch.Size([1, 0, 3072])
1.state.shape=torch.Size([1, 0, 3072])
2.state.shape=torch.Size([1, 0, 3072])
3.state.shape=torch.Size([1, 0, 3072])
4.state.shape=torch.Size([1, 0, 3072])
1.state.shape=torch.Size([1, 0, 3072])
2.state.shape=torch.Size([1, 0, 3072])
3.state.shape=torch.Size([1, 0, 3072])
4.state.shape=torch.Size([1, 0, 3072])
1.state.shape=torch.Size([1, 0, 3072])
2.state.shape=torch.Size([1, 0, 3072])
3.state.shape=torch.Size([1, 0, 3072])
4.state.shape=torch.Size([1, 0, 3072])


tensor([], device='cuda:0', size=(4, 1, 0, 128256), dtype=torch.bfloat16)